In [1]:
import create_population
import preprocessing
from feature_engineering import FeatureCreator
from feature_engineering_prediction import PredictionFeatureCreator

import prediction

import pandas as pd

from train_gbm import Trainer_gbm
from train_gbm_time import Trainer_gbm_time

from evaluation_gbm import Evaluator_gbm
from evaluation_gbm_time import Evaluator_gbm_time
from evaluation_pop import Evaluator_pop

%load_ext autoreload

In [104]:
%autoreload

In [3]:
population = create_population.create(from_="2022-01-01", to_="2023-12-31")

In [4]:
# レース結果テーブルの前処理
results_preprocessed = preprocessing.process_results()

In [60]:
# 馬の過去成績テーブルの加工
horse_results_preprocessed = preprocessing.process_horse_results()
pd.set_option('display.max_columns', None)
horse_results_preprocessed

,horse_id,date,rank,prize,rank_diff,weather,race_type,course_len,ground_state,race_class,n_horses,time,nobori,win,rentai,show,place,corner_1,corner_2,corner_3,corner_4,pase_1,pase_2
16,2019103610,2022-12-04,16.0,0.0,2.5,1.0,0,1200,0.0,5.0,16.0,74.5,39.6,0,0,0,7.0,3,3,<NA>,<NA>,34.4,37.6
17,2019103610,2022-09-25,15.0,0.0,3.1,1.0,0,1200,2.0,3.0,16.0,72.8,39.0,0,0,0,6.0,4,4,<NA>,<NA>,33.2,36.5
18,2019103610,2022-08-14,12.0,0.0,1.9,1.0,0,1000,2.0,3.0,12.0,61.1,38.1,0,0,0,1.0,3,4,<NA>,<NA>,34.4,36.4
19,2019103610,2022-06-18,1.0,770.0,0.0,1.0,0,1000,0.0,2.0,12.0,58.8,35.8,1,1,1,2.0,2,2,<NA>,<NA>,34.4,35.9
20,2019103610,2022-06-12,4.0,120.0,0.4,1.0,0,1000,0.0,2.0,12.0,59.1,36.1,0,0,0,2.0,1,1,<NA>,<NA>,34.5,35.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305459,2021102427,2024-04-06,8.0,0.0,2.3,2.0,0,1800,2.0,1.0,14.0,117.5,39.6,0,0,0,6.0,8,9,6,7,39.1,38.0
305460,2021102427,2023-12-28,13.0,0.0,2.5,1.0,1,1800,0.0,0.0,14.0,110.0,36.0,0,0,0,9.0,9,9,<NA>,<NA>,36.0,34.1
305469,2021103807,2024-02-24,11.0,0.0,3.4,2.0,0,1800,2.0,1.0,11.0,118.9,43.0,0,0,0,9.0,1,1,1,2,36.8,39.7
305470,2021103807,2024-01-13,11.0,0.0,2.1,1.0,0,1800,0.0,1.0,14.0,117.9,40.8,0,0,0,8.0,14,12,9,8,35.8,39.8


In [7]:
# レース情報テーブルの前処理
race_info_preprocessed = preprocessing.process_race_info()

In [8]:
# 騎手リーディングテーブルの前処理
jockey_leading_preprocessed = preprocessing.process_jockey_leading()

In [9]:
# 調教師リーディングテーブルの前処理
trainer_leading_preprocessed = preprocessing.process_trainer_leading()

In [10]:
# 血統テーブルの前処理
peds_preprocessed = preprocessing.process_peds()

In [11]:
# 種牡馬リーディングテーブルの前処理
sire_leading_preprocessed = preprocessing.process_sire_leading()

In [12]:
# 払い戻しテーブルの前処理
return_tables_preprocessed = preprocessing.process_return_tables()

# 特徴量作成

In [61]:
fc = FeatureCreator()
features = fc.create_features()

agg_horse_n_races_relative:   0%|          | 0/5 [00:00<?, ?it/s]

running agg_jockey()...
running agg_trainer()...


agg_horse_per_course_len:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_ground_state_race_type:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_race_class:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_race_type:   0%|          | 0/6 [00:00<?, ?it/s]

running agg_sire()...
merging all features...


In [62]:
fc.population

,race_id,date,horse_id
0,202206010101,2022-01-05,2019103610
1,202206010101,2022-01-05,2019100855
2,202206010101,2022-01-05,2019103542
3,202206010101,2022-01-05,2019104288
4,202206010101,2022-01-05,2019101003
...,...,...,...
94887,202309050912,2023-12-28,2017104873
94888,202309050912,2023-12-28,2019100653
94889,202309050912,2023-12-28,2017106240
94890,202309050912,2023-12-28,2018103205


In [63]:
fc.population.race_id.nunique()

6912

In [64]:
fc.horse_results

,horse_id,date,rank,prize,rank_diff,weather,race_type,course_len,ground_state,race_class,n_horses,time,nobori,win,rentai,show,place,corner_1,corner_2,corner_3,corner_4,pase_1,pase_2
0,2019103610,2022-12-04,16.0,0.0,2.5,1.0,0,1200,0.0,5.0,16.0,74.5,39.6,0,0,0,7.0,3,3.0,NaN,NaN,34.4,37.6
1,2019103610,2022-09-25,15.0,0.0,3.1,1.0,0,1200,2.0,3.0,16.0,72.8,39.0,0,0,0,6.0,4,4.0,NaN,NaN,33.2,36.5
2,2019103610,2022-08-14,12.0,0.0,1.9,1.0,0,1000,2.0,3.0,12.0,61.1,38.1,0,0,0,1.0,3,4.0,NaN,NaN,34.4,36.4
3,2019103610,2022-06-18,1.0,770.0,0.0,1.0,0,1000,0.0,2.0,12.0,58.8,35.8,1,1,1,2.0,2,2.0,NaN,NaN,34.4,35.9
4,2019103610,2022-06-12,4.0,120.0,0.4,1.0,0,1000,0.0,2.0,12.0,59.1,36.1,0,0,0,2.0,1,1.0,NaN,NaN,34.5,35.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186195,2021102427,2024-04-06,8.0,0.0,2.3,2.0,0,1800,2.0,1.0,14.0,117.5,39.6,0,0,0,6.0,8,9.0,6.0,7.0,39.1,38.0
186196,2021102427,2023-12-28,13.0,0.0,2.5,1.0,1,1800,0.0,0.0,14.0,110.0,36.0,0,0,0,9.0,9,9.0,NaN,NaN,36.0,34.1
186197,2021103807,2024-02-24,11.0,0.0,3.4,2.0,0,1800,2.0,1.0,11.0,118.9,43.0,0,0,0,9.0,1,1.0,1.0,2.0,36.8,39.7
186198,2021103807,2024-01-13,11.0,0.0,2.1,1.0,0,1800,0.0,1.0,14.0,117.9,40.8,0,0,0,8.0,14,12.0,9.0,8.0,35.8,39.8


In [71]:
df = (
    fc.population
    .merge(fc.horse_results,on="horse_id",suffixes=("","_horse"))
    .query("date_horse < date")
    .sort_values("date_horse",ascending = False)#dateの昇順
    .groupby(["race_id","horse_id"])#レースとホースに起因して
    .head(5)#直近5レース
)
df

,race_id,date,horse_id,date_horse,rank,prize,rank_diff,weather,race_type,course_len,ground_state,race_class,n_horses,time,nobori,win,rentai,show,place,corner_1,corner_2,corner_3,corner_4,pase_1,pase_2
1523411,202309050903,2023-12-28,2021102811,2023-12-23,17.0,0.0,3.3,1.0,1,1400,0.0,0.0,17.0,84.3,35.9,0,0,0,9.0,13,15.0,NaN,NaN,34.8,34.8
1523347,202309050903,2023-12-28,2021105731,2023-12-23,12.0,0.0,1.6,1.0,1,1800,0.0,1.0,14.0,109.9,35.0,0,0,0,9.0,6,8.0,NaN,NaN,35.4,34.4
1523405,202309050903,2023-12-28,2021105018,2023-12-23,14.0,0.0,3.3,1.0,0,1200,0.0,1.0,15.0,76.9,40.2,0,0,0,9.0,10,12.0,NaN,NaN,35.6,38.0
1523388,202309050903,2023-12-28,2021104215,2023-12-23,14.0,0.0,2.0,1.0,1,1800,0.0,1.0,14.0,110.3,34.7,0,0,0,9.0,14,14.0,NaN,NaN,35.4,34.4
1511977,202306050704,2023-12-23,2021103822,2023-12-17,9.0,0.0,4.2,1.0,0,1800,0.0,0.0,16.0,120.9,42.1,0,0,0,6.0,14,14.0,12.0,13.0,38.3,40.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156182,202206020404,2022-03-06,2013106297,2015-10-25,8.0,0.0,1.2,1.0,1,1000,0.0,1.0,17.0,57.6,34.7,0,0,0,4.0,4,NaN,NaN,NaN,33.7,33.8
156183,202206020404,2022-03-06,2013106297,2015-09-05,8.0,0.0,1.3,2.0,0,1000,0.0,1.0,13.0,60.8,37.3,0,0,0,10.0,6,5.0,NaN,NaN,35.0,36.4
259239,202203010304,2022-04-23,2012103700,2015-09-05,14.0,0.0,2.1,1.0,1,1600,0.0,1.0,18.0,96.2,34.6,0,0,0,4.0,13,17.0,NaN,NaN,35.0,35.6
259240,202203010304,2022-04-23,2012103700,2015-08-02,9.0,0.0,1.9,1.0,1,2200,0.0,1.0,14.0,135.5,38.1,0,0,0,4.0,1,1.0,1.0,2.0,34.4,36.2


In [72]:
df.groupby(["race_id", "horse_id"])[["rank", "prize"]].mean()
#レースidとホースidを紐づけて、直近5レースの着順と賞金の平均を出す

rank  prize
race_id      horse_id               
202201010101 2020100678   4.0  110.0
             2020102360   5.5   55.0
             2020102526   6.0    0.0
             2020102535   3.0  180.0
             2020105743   4.5   81.0
...                       ...    ...
202310030812 2020103324   7.2   28.0
             2020103663   6.2   97.0
             2020103754   6.0  145.0
             2020105644   7.6   27.6
             2020106455   7.2   57.6

[85316 rows x 2 columns]

In [77]:
agg_df = df.groupby(["race_id", "horse_id"])[
["rank", "prize","rank_diff","course_len","race_class",
 "n_horses","time","nobori",
 "corner_1","corner_2","corner_3","corner_4","pase_1","pase_2"]
].agg(["mean","median","max","min"])
agg_df 

rank                    prize                       \
                        mean median   max  min   mean median    max    min   
race_id      horse_id                                                        
202201010101 2020100678  4.0    4.0   4.0  4.0  110.0  110.0  110.0  110.0   
             2020102360  5.5    5.5   7.0  4.0   55.0   55.0  110.0    0.0   
             2020102526  6.0    6.0   6.0  6.0    0.0    0.0    0.0    0.0   
             2020102535  3.0    3.0   3.0  3.0  180.0  180.0  180.0  180.0   
             2020105743  4.5    4.5   5.0  4.0   81.0   81.0  110.0   52.0   
...                      ...    ...   ...  ...    ...    ...    ...    ...   
202310030812 2020103324  7.2    7.0  10.0  3.0   28.0    0.0  140.0    0.0   
             2020103663  6.2    5.0  10.0  4.0   97.0   78.0  300.0    0.0   
             2020103754  6.0    5.0  13.0  1.0  145.0   55.0  550.0    0.0   
             2020105644  7.6    8.0  13.0  4.0   27.6    0.0   83.0    0.0   
             2020106455  7.2    7.0  12.0  2.0   57.6    0.0  210.0    0.0   

                        rank_diff                  course_len                \
                             mean median  max  min       mean  median   max   
race_id      horse_id                                                         
202201010101 2020100678      0.70   0.70  0.7  0.7     1800.0  1800.0  1800   
             2020102360      1.45   1.45  2.1  0.8     1200.0  1200.0  1200   
             2020102526      1.20   1.20  1.2  1.2     1800.0  1800.0  1800   
             2020102535      0.80   0.80  0.8  0.8     1800.0  1800.0  1800   
             2020105743      0.80   0.80  1.2  0.4     1200.0  1200.0  1200   
...                           ...    ...  ...  ...        ...     ...   ...   
202310030812 2020103324      0.76   0.90  1.2  0.0     2160.0  2000.0  2600   
             2020103663      1.14   1.30  2.4  0.3     2000.0  2000.0  2200   
             2020103754      1.18   1.30  2.3  0.0     1980.0  2000.0  2200   
             2020105644      1.22   0.80  2.3  0.6     2080.0  2000.0  2400   
             2020106455      1.04   1.10  2.3  0.2     1900.0  2000.0  2000   

                              race_class                  n_horses         \
                          min       mean median  max  min     mean median   
race_id      horse_id                                                       
202201010101 2020100678  1800        0.0    0.0  0.0  0.0     12.0   12.0   
             2020102360  1200        0.5    0.5  1.0  0.0      9.5    9.5   
             2020102526  1800        0.0    0.0  0.0  0.0     11.0   11.0   
             2020102535  1800        0.0    0.0  0.0  0.0      6.0    6.0   
             2020105743  1200        0.5    0.5  1.0  0.0      7.5    7.5   
...                       ...        ...    ...  ...  ...      ...    ...   
202310030812 2020103324  2000        1.0    1.0  1.0  1.0     16.4   16.0   
             2020103663  1800        2.6    2.0  5.0  2.0      9.8   10.0   
             2020103754  1800        1.6    2.0  2.0  1.0     13.6   14.0   
             2020105644  1800        1.0    1.0  1.0  1.0     14.8   15.0   
             2020106455  1700        0.8    1.0  1.0  0.0     15.2   16.0   

                                       time                       nobori  \
                          max   min    mean  median    max    min   mean   
race_id      horse_id                                                      
202201010101 2020100678  12.0  12.0  110.90  110.90  110.9  110.9  34.90   
             2020102360  10.0   9.0   70.35   70.35   71.5   69.2  35.50   
             2020102526  11.0  11.0  115.40  115.40  115.4  115.4  37.30   
             2020102535   6.0   6.0  111.70  111.70  111.7  111.7  36.40   
             2020105743   8.0   7.0   71.60   71.60   72.1   71.1  35.80   
...                       ...   ...     ...     ...    ...    ...    ...   
202310030812 2020103324  18.0  15.0  133.44  124.10  162.0  1

In [79]:
agg_df.columns = ["_".join(col) for col in agg_df.columns]
agg_df

rank_mean  rank_median  rank_max  rank_min  \
race_id      horse_id                                                 
202201010101 2020100678        4.0          4.0       4.0       4.0   
             2020102360        5.5          5.5       7.0       4.0   
             2020102526        6.0          6.0       6.0       6.0   
             2020102535        3.0          3.0       3.0       3.0   
             2020105743        4.5          4.5       5.0       4.0   
...                            ...          ...       ...       ...   
202310030812 2020103324        7.2          7.0      10.0       3.0   
             2020103663        6.2          5.0      10.0       4.0   
             2020103754        6.0          5.0      13.0       1.0   
             2020105644        7.6          8.0      13.0       4.0   
             2020106455        7.2          7.0      12.0       2.0   

                         prize_mean  prize_median  prize_max  prize_min  \
race_id      horse_id                                                     
202201010101 2020100678       110.0         110.0      110.0      110.0   
             2020102360        55.0          55.0      110.0        0.0   
             2020102526         0.0           0.0        0.0        0.0   
             2020102535       180.0         180.0      180.0      180.0   
             2020105743        81.0          81.0      110.0       52.0   
...                             ...           ...        ...        ...   
202310030812 2020103324        28.0           0.0      140.0        0.0   
             2020103663        97.0          78.0      300.0        0.0   
             2020103754       145.0          55.0      550.0        0.0   
             2020105644        27.6           0.0       83.0        0.0   
             2020106455        57.6           0.0      210.0        0.0   

                         rank_diff_mean  rank_diff_median  rank_diff_max  \
race_id      horse_id                                                      
202201010101 2020100678            0.70              0.70            0.7   
             2020102360            1.45              1.45            2.1   
             2020102526            1.20              1.20            1.2   
             2020102535            0.80              0.80            0.8   
             2020105743            0.80              0.80            1.2   
...                                 ...               ...            ...   
202310030812 2020103324            0.76              0.90            1.2   
             2020103663            1.14              1.30            2.4   
             2020103754            1.18              1.30            2.3   
             2020105644            1.22              0.80            2.3   
             2020106455            1.04              1.10            2.3   

                         rank_diff_min  course_len_mean  course_len_median  \
race_id      horse_id                                                        
202201010101 2020100678            0.7           1800.0             1800.0   
             2020102360            0.8           1200.0             1200.0   
             2020102526            1.2           1800.0             1800.0   
             2020102535            0.8           1800.0             1800.0   
             2020105743            0.4           1200.0             1200.0   
...                                ...              ...                ...   
202310030812 2020103324            0.0           2160.0             2000.0   
             2020103663            0.3           2000.0             2000.0   
             2020103754            0.0           1980.0             2000.0   
             2020105644            0.6           2080.0             2000.0   
             2020106455            0.2           1900.0             2000.0   

                         course_len_max  course_len_min  race_class_mean  \
race_id      horse_id                        

In [85]:
#相対値が意味ある＿標準化
relative_df = (agg_df -  agg_df.groupby("race_id").mean()) / agg_df.groupby("race_id").std()
relative_df.head(10)

rank_mean  rank_median  rank_max  rank_min  \
race_id      horse_id                                                 
202201010101 2020100678  -0.502625    -0.502625 -0.632456 -0.182574   
             2020102360   0.753937     0.753937  1.264911 -0.182574   
             2020102526   1.172791     1.172791  0.632456  1.643168   
             2020102535  -1.340333    -1.340333 -1.264911 -1.095445   
             2020105743  -0.083771    -0.083771  0.000000 -0.182574   
202201010102 2019100025   1.406676     1.692228  1.218015  0.046909   
             2019101760  -0.354142    -0.564076 -0.064106  0.046909   
             2019102309  -0.908107    -0.564076 -1.025697 -0.891278   
             2019102321  -0.195866    -1.208734  0.897485 -0.891278   
             2019102757  -0.275004    -0.241747 -0.705167  0.516003   

                         prize_mean  prize_median  prize_max  prize_min  \
race_id      horse_id                                                     
202201010101 2020100678    0.371864      0.371864   0.123886   0.539179   
             2020102360   -0.452834     -0.452834   0.123886  -0.886534   
             2020102526   -1.277532     -1.277532  -1.579546  -0.886534   
             2020102535    1.421479      1.421479   1.207888   1.446450   
             2020105743   -0.062977     -0.062977   0.123886  -0.212561   
202201010102 2019100025   -0.801467     -0.817424  -0.489476  -0.316228   
             2019101760   -0.199209      0.350325  -0.489476  -0.316228   
             2019102309    0.803647      0.350325   1.256763  -0.316228   
             2019102321    1.664795      2.101947   1.256763  -0.316228   
             2019102757   -0.766040     -0.817424  -0.833432  -0.316228   

                         rank_diff_mean  rank_diff_median  rank_diff_max  \
race_id      horse_id                                                      
202201010101 2020100678       -0.903607         -0.903607      -0.905357   
             2020102360        1.433307          1.433307       1.629643   
             2020102526        0.654336          0.654336       0.000000   
             2020102535       -0.592018         -0.592018      -0.724286   
             2020105743       -0.592018         -0.592018       0.000000   
202201010102 2019100025        2.670426          2.550765       2.682846   
             2019101760       -0.042576         -0.059091       0.000000   
             2019102309       -0.300015         -0.551517       0.062392   
             2019102321       -0.121788          0.039394       0.311959   
             2019102757       -0.854497         -1.043942      -0.623918   

                         rank_diff_min  course_len_mean  course_len_median  \
race_id      horse_id                                                        
202201010101 2020100678      -0.279372         0.730297           0.730297   
             2020102360       0.069843        -1.095445          -1.095445   
             2020102526       1.466704         0.730297           0.730297   
             2020102535       0.069843         0.730297           0.730297   
             2020105743      -1.327018        -1.095445          -1.095445   
202201010102 2019100025       2.015986         0.400005           0.296319   
             2019101760      -0.132196         0.536592           0.296319   
             2019102309      -0.297441        -0.321955          -0.444478   
             2019102321      -1.123664        -0.165856          -0.074080   
             2019102757      -0.462685         0.926840           1.037116   

                         course_len_max  course_len_min  race_class_mean  \
race_id      horse_id                                                      
202201010101 2020100678        0.730297        0.730297        -0.730297   
             2020102360       -1.095445       -1.095445         1.095445   
             2020102526        0.730297        0.730297        -0.730297   
             2020102535        0.730297  

In [86]:
agg_df.head(10)

rank_mean  rank_median  rank_max  rank_min  \
race_id      horse_id                                                 
202201010101 2020100678       4.00          4.0       4.0       4.0   
             2020102360       5.50          5.5       7.0       4.0   
             2020102526       6.00          6.0       6.0       6.0   
             2020102535       3.00          3.0       3.0       3.0   
             2020105743       4.50          4.5       5.0       4.0   
202201010102 2019100025      10.25         12.0      13.0       4.0   
             2019101760       5.80          5.0       9.0       4.0   
             2019102309       4.40          5.0       6.0       2.0   
             2019102321       6.20          3.0      12.0       2.0   
             2019102757       6.00          6.0       7.0       5.0   

                         prize_mean  prize_median  prize_max  prize_min  \
race_id      horse_id                                                     
202201010101 2020100678       110.0         110.0      110.0      110.0   
             2020102360        55.0          55.0      110.0        0.0   
             2020102526         0.0           0.0        0.0        0.0   
             2020102535       180.0         180.0      180.0      180.0   
             2020105743        81.0          81.0      110.0       52.0   
202201010102 2019100025        19.5           0.0       78.0        0.0   
             2019101760        41.6          52.0       78.0        0.0   
             2019102309        78.4          52.0      210.0        0.0   
             2019102321       110.0         130.0      210.0        0.0   
             2019102757        20.8           0.0       52.0        0.0   

                         rank_diff_mean  rank_diff_median  rank_diff_max  \
race_id      horse_id                                                      
202201010101 2020100678            0.70              0.70            0.7   
             2020102360            1.45              1.45            2.1   
             2020102526            1.20              1.20            1.2   
             2020102535            0.80              0.80            0.8   
             2020105743            0.80              0.80            1.2   
202201010102 2019100025            4.20              4.05            6.8   
             2019101760            1.46              1.40            2.5   
             2019102309            1.20              0.90            2.6   
             2019102321            1.38              1.50            3.0   
             2019102757            0.64              0.40            1.5   

                         rank_diff_min  course_len_mean  course_len_median  \
race_id      horse_id                                                        
202201010101 2020100678            0.7           1800.0             1800.0   
             2020102360            0.8           1200.0             1200.0   
             2020102526            1.2           1800.0             1800.0   
             2020102535            0.8           1800.0             1800.0   
             2020105743            0.4           1200.0             1200.0   
202201010102 2019100025            1.9           1825.0             1800.0   
             2019101760            0.6           1860.0             1800.0   
             2019102309            0.5           1640.0             1600.0   
             2019102321            0.0           1680.0             1700.0   
             2019102757            0.4           1960.0             2000.0   

                         course_len_max  course_len_min  race_class_mean  \
race_id      horse_id                                                      
202201010101 2020100678            1800            1800             0.00   
             2020102360            1200            1200             0.50   
             2020102526            1800            1800             0.00   
             2020102535            1800  

In [87]:
(72.5+180+70)/6

53.75

In [88]:
(70.0 -53.75) / 71.023764

0.22879666022769507

In [90]:
fc = FeatureCreator()
features = fc.create_features()

agg_horse_n_races_relative:   0%|          | 0/5 [00:00<?, ?it/s]

running agg_jockey()...
running agg_trainer()...


agg_horse_per_course_len:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_ground_state_race_type:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_race_class:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_race_type:   0%|          | 0/6 [00:00<?, ?it/s]

running agg_sire()...
merging all features...


In [91]:
features

race_id        date    horse_id  jockey_id  trainer_id  owner_id  \
0      202206010101  2022-01-05  2019103610       1192        1020    897009   
1      202206010101  2022-01-05  2019100855       1179        1088    125008   
2      202206010101  2022-01-05  2019103542       1009        1089    546800   
3      202206010101  2022-01-05  2019104288       1119        1132     64803   
4      202206010101  2022-01-05  2019101003       5212        1109    427030   
...             ...         ...         ...        ...         ...       ...   
94108  202309050912  2023-12-28  2017104873       1154         425    758005   
94109  202309050912  2023-12-28  2019100653       1138        1158    950006   
94110  202309050912  2023-12-28  2017106240       1018        1178    258033   
94111  202309050912  2023-12-28  2018103205       1112         428    629800   
94112  202309050912  2023-12-28  2017102603       1144        1050    356002   

       rank  time  nobori  umaban  wakuban  tansho_odds  popularity  impost  \
0         1  72.5    37.6      15        8          6.8           4    51.0   
1         2  72.5    37.7      10        5         57.2          12    54.0   
2         3  72.5    37.3       4        2          3.7           1    56.0   
3         4  72.8    37.2       2        1         16.0           9    54.0   
4         5  73.2    38.0      16        8         10.0           5    56.0   
...     ...   ...     ...     ...      ...          ...         ...     ...   
94108    12  69.5    33.6      11        6        172.3          13    58.0   
94109    13  69.5    34.2      15        8         94.7          12    56.0   
94110    14  69.6    34.5       7        4         75.0          10    58.0   
94111    15  69.8    34.9      16        8        368.3          15    58.0   
94112    16  70.4    34.4      14        7        347.8          14    58.0   

       sex  age  weight  weight_diff  corner_1  corner_2  corner_3  corner_4  \
0        1    3     456            4         2       2.0       NaN       NaN   
1        1    3     458            2         1       1.0       NaN       NaN   
2        0    3     518           -2         4       3.0       NaN       NaN   
3        1    3     486            6        10       7.0       NaN       NaN   
4        0    3     478           -2         4       5.0       NaN       NaN   
...    ...  ...     ...          ...       ...       ...       ...       ...   
94108    0    6     512            4        15      15.0       NaN       NaN   
94109    1    4     498           10         9      11.0       NaN       NaN   
94110    0    6     508            2         5       6.0       NaN       NaN   
94111    0    5     472           -4         5       4.0       NaN       NaN   
94112    0    6     490           14        16      15.0       NaN       NaN   

       race_type  around  course_len  weather  ground_state  race_class  \
0              0     0.0      1200.0        1             0           1   
1              0     0.0      1200.0        1             0           1   
2              0     0.0      1200.0        1             0           1   
3              0     0.0      1200.0        1             0           1   
4              0     0.0      1200.0        1             0           1   
...          ...     ...         ...      ...           ...         ...   
94108          1     0.0      1200.0        1             0           4   
94109          1     0.0      1200.0        1             0           4   
94110          1     0.0      1200.0        1             0           4   
94111          1     0.0      1200.0        1             0           4   
94112          1     0.0      1200.0        1             0           4   

       place  rank_3races  prize_3races  rank_5races  prize_5races  \
0          6     9.666667     43.333333     9.250000     32.500000   
1          6    11.000000      0.000000    11.000000      0.000000   
2          6     3.333333    1

In [92]:
features.columns

Index(['race_id', 'date', 'horse_id', 'jockey_id', 'trainer_id', 'owner_id',
       'rank', 'time', 'nobori', 'umaban',
       ...
       'win_mean_20races_per_race_type_relative',
       'win_max_20races_per_race_type_relative',
       'win_min_20races_per_race_type_relative',
       'show_mean_20races_per_race_type_relative',
       'show_max_20races_per_race_type_relative',
       'show_min_20races_per_race_type_relative', 'sire_n_races_relative',
       'sire_n_wins_relative', 'sire_winrate_relative',
       'sire_course_len_diff_relative'],
      dtype='object', length=749)

In [102]:
import yaml
feature_cols = {
    "features": features.columns.drop(
        ["rank","time","nobori","nobori","corner_1","corner_2","corner_3","corner_4","date","owner_id"]
    ).to_list()
}
with open("config_new.yaml","w") as file:
    yaml.dump(feature_cols,file
             )

In [105]:
trainer = Trainer_gbm(config_filepath="config_new.yaml")
evaluation_df = trainer.run(
    valid_start_date="2022-10-01",
    test_start_date="2023-01-01",
    importance_filename="importance_gbm_new",
    model_filename="model_gbm_new.pkl",
    evaluation_filename="evaluation_gbm_new.csv"
)

Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.214688
[200]	valid_0's binary_logloss: 0.20661
[300]	valid_0's binary_logloss: 0.204652
[400]	valid_0's binary_logloss: 0.204184
[500]	valid_0's binary_logloss: 0.204177
Early stopping, best iteration is:
[446]	valid_0's binary_logloss: 0.204107


,race_id,horse_id,target,rank,tansho_odds,popularity,umaban,pred
46840,202306010101,2020103575,1,1,1.8,1,14,0.322221
46841,202306010101,2020107073,0,2,7.4,3,5,0.072154
46842,202306010101,2020102562,0,3,36.1,6,1,0.015019
46843,202306010101,2020106345,0,4,4.2,2,15,0.311751
46844,202306010101,2020100039,0,5,7.9,4,9,0.059590
...,...,...,...,...,...,...,...,...
94108,202309050912,2017104873,0,12,172.3,13,11,0.010730
94109,202309050912,2019100653,0,13,94.7,12,15,0.006823
94110,202309050912,2017106240,0,14,75.0,10,7,0.007850
94111,202309050912,2018103205,0,15,368.3,15,16,0.009785


-------------------- result --------------------
test_df's binary_logloss: 0.2074270068148725


In [106]:
evaluator = Evaluator_gbm(evaluation_filename="evaluation_gbm_new.csv")
summary_df = evaluator.summarize_box_exp(exp_name="gbm_new")
summary_df

,topn,bet_type,hitrate_gbm_new,returnrate_gbm_new
0,1,単勝,0.333044,0.808652
1,1,複勝,0.635127,0.844647
0,2,ワイド,0.301794,0.782350
1,2,単勝,0.526620,0.807205
2,2,複勝,0.837384,0.833420
3,2,馬単,0.146412,0.728501
4,2,馬連,0.146412,0.752459
0,3,ワイド,0.539062,0.809915
1,3,三連単,0.090856,0.721330
2,3,三連複,0.090856,0.847367
